# Generating texts
In this notebook we will generate texts and store them in a CSV file. More precisely, we generate random PhD topics which could potentially be worked on at the UFZ Leipzig structured as described in its [organigram](https://www.ufz.de/export/data/global/98993_organisation-chart_27-10-2025.jpg).

In [1]:
import bia_bob
import random

First, we test the connection to the LLM server.

In [2]:
def prompt_scadsai_llm(message:str, model="openai/gpt-oss-120b"):
    """A prompt helper function that sends a message to ScaDS.AI LLM server at 
    ZIH TU Dresden and returns only the text response.
    """
    import os
    import openai
    
    # convert message in the right format if necessary
    if isinstance(message, str):
        message = [{"role": "user", "content": message}]
    
    # setup connection to the LLM
    client = openai.OpenAI(base_url="https://llm.scads.ai/v1",
                           api_key=os.environ.get('SCADSAI_API_KEY')
    )
    response = client.chat.completions.create(
        model=model,
        messages=message
    )
    
    # extract answer
    return response.choices[0].message.content
     
prompt_scadsai_llm("hello world")

'Hello! How can I help you today?'

Next, we define a couple of random first and lastnames. We also define a list of institutes.
These will hint the LLM to generate PhD topics that are relevant to a given institute.

In [3]:
import pandas as pd
import numpy as np
import random

# Define separate lists for first names and last names
first_names = ["Alex", "Jordan", "Morgan", "Taylor", "Avery",
               "Sam", "Casey", "Riley", "Jamie", "Dakota",
               "Bailey", "Cameron", "Sydney", "Robin", "Charlie",
               "Dana", "Kris", "Skyler", "Devon", "Reese"]

last_names = ["Reed", "Smith", "Lee", "Garcia", "Patel",
              "Chen", "O'Hara", "Jain", "Kumar", "Singh",
              "Liu", "Davis", "Clark", "Thomas", "Brooks",
              "Flores", "Rivera", "Adams", "Gonzalez", "Campbell"]

faculties = """Biodiversity & People
Biodiversity Conservation
Biodiversity Economics
Biodiversity in the Anthropocene
Biodiversity Synthesis
Experimental Interaction Ecology
Physiological Diversity
Species Interaction Ecology
Theory in Biodiversity Science""".split("\n")
len(faculties), faculties[:10]

(9,
 ['Biodiversity & People',
  'Biodiversity Conservation',
  'Biodiversity Economics',
  'Biodiversity in the Anthropocene',
  'Biodiversity Synthesis',
  'Experimental Interaction Ecology',
  'Physiological Diversity',
  'Species Interaction Ecology',
  'Theory in Biodiversity Science'])

In the following loop we select an institute name randomly and ask the LLM server to generate a PhD topic related to the chosen institute.

In [4]:
research_fields = faculties

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

profiles = []

#for field in faculties:
while len(profiles) < 250:    
    # Random name generation from first and last name
    name = f"{random.choice(first_names)} {random.choice(last_names)}"
    field = random.choice(research_fields)

    topic = prompt_scadsai_llm(f"""Generate an English PhD thesis topic for someone working at a the Centre for Integrative Biodiversity Research in group '{field}'. Reply with only the title of the research project and nothing else.""")
    topic = topic.strip().replace("</s>", "").replace("\"", "")
    if ":" in topic and " " not in topic.split(":")[0]:
        topic = topic.split(":")[1]
    if "</thinking>" in topic:
        topic = topic.split("</thinking>")[1]

    print(len(topic), topic)
    
    profiles.append({
        "name": name,
        "research_field": field,
        "topic": topic
    })

106 Integrative Modeling of Multi‑Taxon Functional Traits to Predict Ecosystem Resilience Under Climate Change
105 Quantifying the Economic Valuation of Pollinator-Driven Ecosystem Services under Climate Change Scenarios
112 Integrative Landscape Genomics for Enhancing Adaptive Capacity of Threatened Plant Species under Climate Change】
133 Integrating Traditional Ecological Knowledge and Genomic Data to Model Human-Driven Resilience in Agro‑Forest Biodiversity Landscapes
161 Integrating Genomic, Functional, and Landscape Approaches to Assess Resilience of Pollinator Communities Under Climate‑Driven Land‑Use Change in the Anthropocene
116 Assessing the Socio‑Ecological Impacts of Urban Green Infrastructure on Biodiversity Resilience and Human Well‑Being
92 Integrative Theoretical Models of Multiscale Species Coexistence in Heterogeneous Ecosystems
148 Network Dynamics of Mutualistic Interactions under Climate Change: Integrating Trait‑ and Phylogeny‑Based Approaches in Plant‑Pollinator 

Finally, we store the names, institutes and PhD topics to a csv file.

In [5]:
# Create DataFrame and save CSV
df = pd.DataFrame(profiles)
df.to_csv("phd_topics.csv", index=False)

df.head()

,name,research_field,topic
0,Taylor Reed,Biodiversity Synthesis,Integrative Modeling of Multi‑Taxon Functional...
1,Riley Jain,Biodiversity Economics,Quantifying the Economic Valuation of Pollinat...
2,Taylor Adams,Biodiversity Conservation,Integrative Landscape Genomics for Enhancing A...
3,Devon Thomas,Biodiversity & People,Integrating Traditional Ecological Knowledge a...
4,Alex Lee,Biodiversity in the Anthropocene,"Integrating Genomic, Functional, and Landscape..."
